In [ ]:
import numpy as np
import os
import scipy
from experimenting.utils import get_joints_from_heatmap
import cv2

In [ ]:
i = indexes[0]
x = d._get_x(i)
y = np.load(d.labels[i])

homedir = '/data/dhp19'
P_mat_dir = os.path.join(homedir, 'P_matrices/')
P_mat_cam3 = np.load(os.path.join(P_mat_dir,'P3.npy'))
y['xyz']


def _get_heatmap(joints, mask, heigth, width, num_joints):
    u, v = joints
    # initialize, fill and smooth the heatmaps
    label_heatmaps = np.zeros((heigth, width, num_joints))
    for fmidx, zipd in enumerate(zip(v, u, mask)):
        if zipd[2] == 1:  # write joint position only when projection within frame boundaries
            label_heatmaps[zipd[0], zipd[1], fmidx] = 1
            label_heatmaps[:, :, fmidx] = decay_heatmap(label_heatmaps[:, :, fmidx])
    return label_heatmaps

def decay_heatmap(heatmap, sigma2=4):
    heatmap = cv2.GaussianBlur(heatmap, (0, 0), sigma2)
    heatmap /= np.max(heatmap)  # keep the max to 1
    return heatmap

def get_joints2d(vicon_xyz, p_mat, width, heigth, swap='xy'):
    num_joints = vicon_xyz.shape[-1]
    vicon_xyz_homog = np.concatenate([vicon_xyz, np.ones([1, num_joints])], axis=0)
    coord_pix_homog = np.matmul(p_mat, vicon_xyz_homog)
    print(coord_pix_homog.shape)
    coord_pix_homog_norm = coord_pix_homog / coord_pix_homog[-1]

    u = coord_pix_homog_norm[0]
    v = heigth - coord_pix_homog_norm[1]  # flip v coordinate to match the image direction

    # mask is used to make sure that pixel positions are in frame range.
    mask = np.ones(u.shape).astype(np.float32)
    mask[np.isnan(u)] = 0
    mask[np.isnan(v)] = 0
    mask[u > width] = 0
    mask[u <= 0] = 0
    mask[v > heigth] = 0
    mask[v <= 0] = 0

    # pixel coordinates
    u = u.astype(np.int32)
    v = v.astype(np.int32)
    return u, v, coord_pix_homog
u, v, c = get_joints2d(y['xyz'], P_mat_cam3, 256, 256)
X = (u, v)
X = y['xyz'][:, 0]
x = (u[0], v[0])
def get_projection_components(P):
        Q = P[:3, :3]
        q = P[:, 3]
        U, S =scipy.linalg.qr(np.linalg.inv(Q))
        R = np.linalg.inv(U)
        K = np.linalg.inv(S)
        t = S.dot(q)        
        K = K / K[2, 2]

        return K, R, t
K, R, t = get_projection_components(P_mat_cam3)
Xt = np.concatenate([y['xyz'], np.ones((1, 13))], axis=0)
M = np.concatenate([R, np.expand_dims(t, 1)], axis=1)
P = K.dot(M)
Xt = np.concatenate([X, np.array([1])])
a = P.dot(Xt)
b = a / a[2]
b
Xt = np.concatenate([X, np.array([1])])
